In [5]:
import sys
sys.path.append('..')

import random

# Pynetics QuickStart

In this example we are going to build a very simple and useless algorithm to explore the possibilities of the pynetics library.

Our problem will be as follows. We'll going to develop a genetic algorithm to find the what binary list of lenght $L=N$ is the one with the bigger sum. Yes, total absurd, but useful to learn GAs and this library.

Let's start from the begining. The individuals.

## Representing individuals

The individuals are the most important component in a genetic algorithm. Each individual is a possible solution, good or bad, for our problem.

We want to model an individual capable of represent a possible solution for our problem. Pynetics have a perfect representation for this problem, the `BinaryIndividual`, so it's no necessary to create a custom individual. We'll cross that bridge when we get to it.

In [17]:
from pynetics.ga_bin import BinaryIndividual

# Let's define the size of our individuals (the numer of 1's and 0's)
individual_size = 25

# We create an individual with random 0's and 1's
individual = BinaryIndividual()
individual.extend(random.getrandbits(1) for _ in range(individual_size))

print(individual)

1001111001001000101001110


## Fitness

Our individuals are solutions for the problem but, ¿how can we measure how good or how bad are they? That is what the `fitness` is for. It's a function that will return a float value. The bigger the value, the better the individual is.

We could use a fitness function equals to the sum of al $1$'s but if we want to stop the algorithm based on the fitness, is not the same the best fitness for an individual of size 10 than an individual of size 20.

So the fitness funcion we're gonna use is a function with the form $1 / (1 + \alpha)$, being $\alpha$ the error of our individual. The error will be computed as the number of $0$'s the individual has.

In [49]:
def maximize_ones_fitness(individual):
    error = len(individual) - sum(individual)
    return 1 / (1 + error)

This function guarantees that the fitness will belong to the $(0, 1]$ interval. Let's see an example of how it behaves.

In [50]:
# Let's create some individuals first
individuals = []
for _ in range(10):
    individual = BinaryIndividual()
    individual.extend(random.getrandbits(1) for _ in range(individual_size))
    individuals.append(individual)

# Now check against the fitness function to see how good are the solutions
for individual in individuals:
    fitness = maximize_ones_fitness(individual)
    print('{} -> {}'.format(individual, fitness))

1111100000011010100000100 -> 0.0625
0000111000100011011010011 -> 0.06666666666666667
1100100010001111101011111 -> 0.09090909090909091
0001111011010011110011110 -> 0.09090909090909091
0111000110000110001100101 -> 0.06666666666666667
0100111000101010100010100 -> 0.0625
0111110110000011111011101 -> 0.1
1001011101101001100010100 -> 0.07142857142857142
1111010110011001001110111 -> 0.1
1110110000111001101101010 -> 0.08333333333333333


## Creating individuals

Nice, now we have a representation (i.e. `BinaryIndividuals`) and a fitness function (i.e. `maximize_ones_fitness`). The next part will be create the `SpawningPool`, i.e., the object that will create the individuals in the algorithm.

The same module that contains the `BinaryIndividual` contains the `BinaryIndividualSpawningPool` class that constructs BinaryIndividuals. It should be created pointing out the size of the individuals to be created and their fitness.

In [51]:
from pynetics.ga_bin import BinaryIndividualSpawningPool, AverageHamming

population_size = 10

# TODO Remove from here the diversity and the fitness, and remove the fitness function from the Individual.
binary_individual_spawning_pool=BinaryIndividualSpawningPool(
    size=individual_size,
    fitness=maximize_ones_fitness,
    diversity=AverageHamming(),
)

Now the spawning pool will be capable of creating individuals of the specified size. Let's see an example:

In [52]:
for _ in range(10):
    individual = binary_individual_spawning_pool.spawn()
    print(individual)

0010100010100011001100010
0010100101011010101011110
1010111001011001101100001
1110111010011000001111101
0101000111001100110110100
1010010001000011000110010
1001101001100101001000110
1001110000000111101100010
1100100110001111110101010
1010010111110110100110100


## The stop condition

Now we're gonna specify when our algorithm should stop. This is controlled by a stop condition.

In [53]:
from pynetics.stop import FitnessBound

fitness_stop_condition = FitnessBound(1)

Instances of the class FitnessBound are created by specifying the fitness thresshold above which we can stop our algorithm. We have specified a FitnessBound object with a thressholdd of $1$. That means that all the values below $1$ will not stop our algorithm whereas all the values upper or equal than $1$ will do.

Because our fitness value belongs to the $(0, 1]$ interval, the algorithm will stop only when the population have an individual with a fitness of $1$ (all $1$'s).

## Selecting individuals

For our GA, we're going to use a tournament selection. Tournament selection works by selecting $n$ individuals randomly from the population and then returning the best of then  (based on their fitnesses).

In [54]:
from pynetics.selections import Tournament

tournament_selection = Tournament(2)

## Recombining

Now the recombination, i.e. the step where the individuals are selected and their genetic informatio is going to be inherited by their progeny.

We'll use a `OnePointRecombination`, included in the module `ga_list`. Also, for the recommender we'll going to specify the probability for two individuals to mate to be 1, that is, they always mate.

In [55]:
from pynetics.ga_list import OnePointRecombination

recombination_probability = 1
recombination = OnePointRecombination()

## Mutations

The same with mutations. The mutation operator we're gona use is `AllGenesCanSwitch`, a mutation where for each binary gene there is a probability to be switched from $0$ to $1$ and viceversa. It belongs to the module `ga_bin`.

In [56]:
from pynetics.ga_bin import AllGenesCanSwitch

mutation_probability = 1 / individual_size
mutation = AllGenesCanSwitch()

## Replacement

Once we've got the offspring, we need to replace the population with these new borns. The operator for that matter will be a `LowElitism` operator, where the worst individuals of the popularin are replaced by the offspring.

We'll fix the replacement rate in $0.9$, i.e. a $90\%$ of the pooulation will be replaced for each iteration of the loop.

In [61]:
from pynetics.replacements import LowElitism

replacement_rate = 0.9
replacement = LowElitism()

## The algorithm

In [62]:
from pynetics.algorithms import SimpleGA

ga = SimpleGA(
    stop_condition=fitness_stop_condition,
    size=population_size,
    spawning_pool=binary_individual_spawning_pool,
    selection=tournament_selection,
    recombination=recombination,
    mutation=mutation,
    replacement=replacement,
    p_recombination=recombination_probability,
    p_mutation=mutation_probability,
    replacement_rate=replacement_rate,
)

In [63]:
ga.run()
print(ga.best())

1111111111111111111111111


In [64]:
ga.on_step_end(lambda ga: print(ga.best()))
ga.run()

1110000111111110111100001
1111110100110110010101111
0111110111111101111100001
0111110111111111111100001
0111110111111111111100001
0111110111111111111100001
0111110111111111111100001
0111110111111111111100001
0111110111111111111110011
0111110111111111111110011
0111110111111111111111011
0111110111111111111111011
0111110111111111111111011
0111110111111111111111011
0111110111111111111111011
0111110111111111111111011
0111110111111111111111011
0111111111111111111111011
0111111111111111111111011
0111111111111111111111011
0111111111111111111111011
0111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111011
1111111111111111111111111


And here ends the quickstart tutorial.